In [1]:
import os
import numpy as np
import sys

import json
import pandas as pd
import pickle

In [2]:
# 获取当前目录路径
current_dir = os.getcwd()

# 定义子目录名
subdir_name = 'Output_treads'

# 创建子目录
subdir_path = os.path.join(current_dir, subdir_name)
if not os.path.exists(subdir_path):
    os.makedirs(subdir_path)

In [24]:
DMG_input = 'Output_preprocessing/DMG_sample_R.csv' #pelicun output 经过前处理，去掉了倒塌和不可修复的情况，和无损伤的样本
DL_summary_input = 'Output_pelicun/DL_summary.csv' #pelicun output
IF_delays_input = 'IF_delays_input.csv'
input_params_json = 'Output_preprocessing/input_parameters_R.json'

output_path = subdir_path

In [9]:
pickle_file_path = 'Output_treads/RC_results.pkl'

# 从pickle文件中加载字典数据
with open(pickle_file_path, "rb") as file:
    RC_results = pickle.load(file)

RCmax_RS = np.squeeze(RC_results["RCmax_RS"])

In [16]:
with open(input_params_json) as f:
    input_param = json.load(f)

In [17]:
#impeding factor calculation
total_cost = input_param['total_cost']
coeff_financing = input_param['financing_coeff']
n_structural = input_param['stabilization']['limit_structural']
t_structural = input_param['stabilization']['time_structural']
n_facade = input_param['stabilization']['limit_facade']
t_facade = input_param['stabilization']['time_facade']

In [7]:
DMG = pd.read_csv(DMG_input, low_memory=False, header=None)
PG = DMG.loc[1, 1:] #performance groups indicating the location of fragility

FG = DMG.loc[0, 1:] #fragility groups
n_PG = len(FG)
DL_summary=pd.read_csv(DL_summary_input)
IF_delays=pd.read_csv(IF_delays_input)
med_RC1=IF_delays['Median_RC=1']
med_RC2=IF_delays['Median_RC>1']
disp_RC1=IF_delays['Dispersion_RC=1']
disp_RC2=IF_delays['Dispersion_RC>1']


cases_collapse = DL_summary["collapses/collapsed"]
indx_collapse = cases_collapse[cases_collapse==1].index
cases_irreparable = DL_summary["reconstruction/irreparable"]
indx_irreparable = cases_irreparable[cases_irreparable==1].index
indx_all = DL_summary["#Num"]
indx_repairable =  indx_all.drop(indx_collapse.union(indx_irreparable))

story_FG = []
for i in range(n_PG):
    story_FG.append(int((PG[i+1])))

story = list(set(story_FG))
story.sort()


In [ ]:
story

In [10]:
#%% repairable buildings -- impeding factors
#inspection
IF_inspection = np.zeros(len(indx_repairable))

for i in range (len(RCmax_RS)):
    if (max(RCmax_RS[i,:])>=1):
        IF_inspection[i]= np.random.lognormal(np.log(med_RC2[0]),disp_RC2[0])
IF_inspection_mat2 = np.append(IF_inspection, np.zeros(len(indx_irreparable)+len(indx_collapse)))



In [11]:
#engineering
n1=np.zeros(len(indx_repairable))
n2=np.zeros(len(indx_repairable))
for i in range (len(indx_repairable)):
    x=0
    for j in range(len(story)):
        if ((RCmax_RS[i,x])==1):
            n1[i]=n1[i]+1
        elif ((RCmax_RS[i,x])>1):
            n2[i]=n2[i]+1
        x = x+7
IF_eng = (n1*np.random.lognormal(np.log(med_RC1[1]),disp_RC1[1],len(indx_repairable)) + n2*np.random.lognormal(np.log(med_RC2[1]),disp_RC2[1],len(indx_repairable)))/len(story)
IF_eng_mat2 = np.append(IF_eng, np.zeros(len(indx_irreparable)+len(indx_collapse)))



In [12]:
#permitting
IF_permit = (n1*np.random.lognormal(np.log(med_RC1[2]),disp_RC1[2],len(indx_repairable)) + n2*np.random.lognormal(np.log(med_RC2[2]),disp_RC2[2],len(indx_repairable)))/len(story)
IF_permit_mat2 = np.append(IF_permit, np.zeros(len(indx_irreparable)+len(indx_collapse)))


In [18]:
DL_summary["reconstruction/cost_impractical"] = np.where(DL_summary["repair_cost-"] > total_cost, 1, 0) #维修费用大于重建费用，认为不可修复


In [ ]:
DL_summary

In [19]:
#financing
repair_cost_all = DL_summary[DL_summary["reconstruction/cost_impractical"]==0]
repair_cost = repair_cost_all["repair_cost-"]
ELR = (repair_cost/total_cost).to_numpy()
deductible = 0.1

IF_finance = np.zeros(len(indx_repairable))
rand_num = np.random.rand(len(indx_repairable))
for i in range(len(indx_repairable)):
        #financing is suppoorted by insurance
    if rand_num[i] <=coeff_financing[0]:
        if (ELR[i]>0.05) and (ELR[i]<=deductible):
            IF_finance[i] = 0.5*np.random.lognormal(np.log(med_RC2[4]),disp_RC2[4])
        elif (ELR[i]>deductible):
            IF_finance[i] = max(0.5*np.random.lognormal(np.log(med_RC2[4]),disp_RC2[4]) , np.random.lognormal(np.log(med_RC2[3]),disp_RC2[3]))
        #financing is suppoorted by private loans
    elif coeff_financing[0]< rand_num[i] <=coeff_financing[1]+coeff_financing[0]:
        if (ELR[i]>0.05) and (ELR[i]<=0.1):
            IF_finance[i] = 0.5*np.random.lognormal(np.log(med_RC2[4]),disp_RC2[4])
        elif (ELR[i]>0.1):
            IF_finance[i] = np.random.lognormal(np.log(med_RC2[4]),disp_RC2[4])
    #financing is suppoorted by public loans  
    else: 
        if (ELR[i]>0.1):
            IF_finance[i] = np.random.lognormal(np.log(med_RC2[5]),disp_RC2[5])
        else:
            IF_finance[i] = 0.5*np.random.lognormal(np.log(med_RC2[5]),disp_RC2[5])

IF_finance_mat2 = np.append(IF_finance, np.zeros(len(indx_irreparable)+len(indx_collapse)))

In [21]:
ELR

array([0.        , 0.07502556, 0.        , ..., 0.08179412, 0.08471599,
       0.0771604 ])

In [ ]:
# repair_cost_all = DL_summary[DL_summary["reconstruction/cost_impractical"]==0]
# repair_cost_all

In [21]:
#contractor mobilization for repair sequence 1
IF_cm_rs1 = (n1*np.random.lognormal(np.log(med_RC1[6]),disp_RC1[6],len(indx_repairable)) + n2*np.random.lognormal(np.log(med_RC2[6]),disp_RC2[6],len(indx_repairable)))/len(story)
IF_cm_rs1_mat2 = np.append(IF_cm_rs1, np.zeros(len(indx_irreparable)+len(indx_collapse)))

#contractor mobilization for repair sequence 2
n1=np.zeros(len(indx_repairable))
n2=np.zeros(len(indx_repairable))
for i in range (len(RCmax_RS)):
    x=1
    for j in range(len(story)):
        if ((RCmax_RS[i,x])==1):
            n1[i]=n1[i]+1
        elif ((RCmax_RS[i,x])>1):
            n2[i]=n2[i]+1
        x = x+7
IF_cm_rs2 = (n1*np.random.lognormal(np.log(med_RC1[7]),disp_RC1[7],len(indx_repairable)) + n2*np.random.lognormal(np.log(med_RC2[7]),disp_RC2[7],len(indx_repairable)))/len(story)
IF_cm_rs2_mat2 = np.append(IF_cm_rs2, np.zeros(len(indx_irreparable)+len(indx_collapse)))

#contractor mobilization for repair sequence 3
n1=np.zeros(len(indx_repairable))
n2=np.zeros(len(indx_repairable))
for i in range (len(RCmax_RS)):
    x=2
    for j in range(len(story)):
        if ((RCmax_RS[i,x])==1):
            n1[i]=n1[i]+1
        elif ((RCmax_RS[i,x])>1):
            n2[i]=n2[i]+1
        x = x+7
IF_cm_rs3 = (n1*np.random.lognormal(np.log(med_RC1[8]),disp_RC1[8],len(indx_repairable)) + n2*np.random.lognormal(np.log(med_RC2[8]),disp_RC2[8],len(indx_repairable)))/len(story)  
IF_cm_rs3_mat2 = np.append(IF_cm_rs3, np.zeros(len(indx_irreparable)+len(indx_collapse)))
    
#contractor mobilization for repair sequence 4
n1=np.zeros(len(indx_repairable))
n2=np.zeros(len(indx_repairable))
for i in range (len(RCmax_RS)):
    x=3
    for j in range(len(story)):
        if ((RCmax_RS[i,x])==1):
            n1[i]=n1[i]+1
        elif ((RCmax_RS[i,x])>1):
            n2[i]=n2[i]+1
        x = x+7
IF_cm_rs4 = (n1*np.random.lognormal(np.log(med_RC1[9]),disp_RC1[9],len(indx_repairable)) + n2*np.random.lognormal(np.log(med_RC2[9]),disp_RC2[9],len(indx_repairable)))/len(story) 
IF_cm_rs4_mat2 = np.append(IF_cm_rs4, np.zeros(len(indx_irreparable)+len(indx_collapse)))

#contractor mobilization for repair sequence 5
n1=np.zeros(len(indx_repairable))
n2=np.zeros(len(indx_repairable))
for i in range (len(RCmax_RS)):
    x=4
    for j in range(len(story)):
        if ((RCmax_RS[i,x])==1):
            n1[i]=n1[i]+1
        elif ((RCmax_RS[i,x])>1):
            n2[i]=n2[i]+1
        x = x+7
IF_cm_rs5 = (n1*np.random.lognormal(np.log(med_RC1[10]),disp_RC1[10],len(indx_repairable)) + n2*np.random.lognormal(np.log(med_RC2[10]),disp_RC2[10],len(indx_repairable)))/len(story)
IF_cm_rs5_mat2 = np.append(IF_cm_rs5, np.zeros(len(indx_irreparable)+len(indx_collapse)))

#contractor mobilization for repair sequence 6
n1=np.zeros(len(indx_repairable))
n2=np.zeros(len(indx_repairable))
for i in range (len(RCmax_RS)):
    x=5
    for j in range(len(story)):
        if ((RCmax_RS[i,x])==1):
            n1[i]=n1[i]+1
        elif ((RCmax_RS[i,x])>1):
            n2[i]=n2[i]+1
        x = x+7
IF_cm_rs6 = (n1*np.random.lognormal(np.log(med_RC1[11]),disp_RC1[11],len(indx_repairable)) + n2*np.random.lognormal(np.log(med_RC2[11]),disp_RC2[11],len(indx_repairable)))
IF_cm_rs6_mat2 = np.append(IF_cm_rs6, np.zeros(len(indx_irreparable)+len(indx_collapse)))

#contractor mobilization for repair sequence 7
n1=np.zeros(len(indx_repairable))
n2=np.zeros(len(indx_repairable))
for i in range (len(RCmax_RS)):
    x=6
    for j in range(len(story)):
        if ((RCmax_RS[i,x])==1):
            n1[i]=n1[i]+1
        elif ((RCmax_RS[i,x])>1):
            n2[i]=n2[i]+1
        x = x+7
IF_cm_rs7 = (n1*np.random.lognormal(np.log(med_RC1[12]),disp_RC1[12],len(indx_repairable)) + n2*np.random.lognormal(np.log(med_RC2[12]),disp_RC2[12],len(indx_repairable)))/len(story)
IF_cm_rs7_mat2 = np.append(IF_cm_rs7, np.zeros(len(indx_irreparable)+len(indx_collapse)))

# # Stabilization
# IF_stab_RC5 = np.zeros((len(RCmax_RS))) #structrual repairs
# IF_stab_mob = 0
# N_DMG_RC5_tot=sum(N_DMG_RC5.transpose())
# k1 = (t_structural[0]-t_structural[1])/(n_structural[1]-n_structural[0])
# k2 = (t_facade[0]-t_facade[1])/(n_facade[1]-n_facade[0])
# t1 = k1*n_structural[0]+t_structural[0]
# t2 = k2*n_facade[0]+t_facade[0]
# for i in range(len(RCmax_RS)):
#     if N_DMG_RC5_tot[i]>0 and N_DMG_RC5_tot[i] <= n_structural[0]:
#         IF_stab_RC5[i] = IF_stab_mob + N_DMG_RC5_tot[i]*np.random.lognormal(np.log(t_structural[0]),0.4)
#     elif (N_DMG_RC5_tot[i] >n_structural[0]) and (N_DMG_RC5_tot[i] <n_structural[1]):
#         IF_stab_RC5[i] = IF_stab_mob + N_DMG_RC5_tot[i]*np.random.lognormal(np.log(-k1*N_DMG_RC5_tot[i]+t1),0.4)
#     elif N_DMG_RC5_tot[i] >=n_structural[1]:
#         IF_stab_RC5[i] = IF_stab_mob + (N_DMG_RC5_tot[i]*np.random.lognormal(np.log(t_structural[1]),0.4))/2
        
# IF_stab_RC3 = np.zeros((len(RCmax_RS))) #curtain walls
# N_DMG_RC3_RS_mat = np.squeeze(N_DMG_RC3_RS)[:,np.arange(2,len(np.squeeze(N_DMG_RC3_RS).transpose()),7)]
# N_DMG_RC3_RS3 = sum(N_DMG_RC3_RS_mat.transpose())

# for i in range(len(RCmax_RS)):
#     if N_DMG_RC3_RS3[i]>0 and N_DMG_RC3_RS3[i] <=n_facade[0]:
#         IF_stab_RC3[i] = IF_stab_mob + N_DMG_RC3_RS3[i]*np.random.lognormal(np.log(t_facade[0]),0.4)
#     elif N_DMG_RC3_RS3[i]>n_facade[0] and N_DMG_RC3_RS3[i] <n_facade[1]:
#         IF_stab_RC3[i] = IF_stab_mob + N_DMG_RC3_RS3[i]*np.random.lognormal(np.log(-k2*N_DMG_RC3_RS3[i]+t2),0.4)
#     elif N_DMG_RC3_RS3[i]>= n_facade[1]:
#         IF_stab_RC3[i] = IF_stab_mob + N_DMG_RC3_RS3[i]*np.random.lognormal(np.log(t_facade[1]),0.4)/2

# IF_stab = np.maximum(IF_stab_RC3, IF_stab_RC5) #total stabilization time
# IF_stab_mat2 = np.append(IF_stab, np.zeros(len(indx_irreparable)+len(indx_collapse)))

In [25]:
#%% impeding factor delays for irreparable buildings
IF_reconst_eng = np.random.lognormal(np.log(med_RC2[13]),disp_RC2[13],len(indx_irreparable)+len(indx_collapse))
IF_reconst_insur = np.random.lognormal(np.log(med_RC2[14]),disp_RC2[14],len(indx_irreparable)+len(indx_collapse))
IF_reconst_demol = np.random.lognormal(np.log(med_RC2[15]),disp_RC2[15],len(indx_irreparable)+len(indx_collapse))

IF_reconst = np.maximum(IF_reconst_eng, IF_reconst_insur, IF_reconst_demol)
IF_reconst_mat2 = np.append(np.zeros(len(indx_repairable)),IF_reconst)

IF_matrix = [IF_inspection_mat2.astype(object), IF_eng_mat2.astype(object), IF_permit_mat2.astype(object), IF_finance_mat2.astype(object), IF_cm_rs1_mat2.astype(object), 
                IF_cm_rs2_mat2.astype(object), IF_cm_rs3_mat2.astype(object), IF_cm_rs4_mat2.astype(object), IF_cm_rs5_mat2.astype(object), 
                IF_cm_rs6_mat2.astype(object), IF_cm_rs7_mat2.astype(object), IF_reconst_mat2.astype(object)]


(pd.DataFrame(IF_matrix)).T.to_csv(os.path.join(output_path,r'IF_delays.csv'), header=['IF_inspection','IF_eng','IF_permit','IF_finance',
                                                            'IF_cm_RS1','IF_cm_RS2','IF_cm_RS3','IF_cm_RS4',
                                                            'IF_cm_RS5','IF_cm_RS6','IF_cm_RS7','IF_reconstruct'], index_label='#Num')

In [26]:
#需要保存的变量
# 定义pickle文件的路径
pickle_file_path = os.path.join(subdir_path, 'IF_results.pkl')

output_dict = {
    'IF_inspection': IF_inspection,
    'IF_eng': IF_eng,
    'IF_permit': IF_permit,
    'IF_finance': IF_finance,
    'IF_cm_rs1': IF_cm_rs1,
    'IF_cm_rs2': IF_cm_rs2,
    'IF_cm_rs3': IF_cm_rs3,
    'IF_cm_rs4': IF_cm_rs4,
    'IF_cm_rs5': IF_cm_rs5,
    'IF_cm_rs6': IF_cm_rs6,
    'IF_cm_rs7': IF_cm_rs7,
    'IF_reconst': IF_reconst
}

# 保存数据到pickle文件
with open(pickle_file_path, "wb") as file:
    pickle.dump(output_dict, file)